In [101]:
import os
import numpy as np

smooth_traj_dir = "smoothed_trajs"
raw_traj_dir = "individual_trajs"

# Get all trajectory files in the directories
# Collect all .npz files in each directory
smooth_traj_files = [os.path.join(smooth_traj_dir, f) for f in os.listdir(smooth_traj_dir) if f.endswith('.npz')]
raw_traj_files = [os.path.join(raw_traj_dir, f) for f in os.listdir(raw_traj_dir) if f.endswith('.npz')]

In [106]:
# Load the first file, print its keys, then aggregate from all files under those keys.
merged_data = {}

if len(smooth_traj_files) == 0:
    raise RuntimeError("No trajectory files found!")

# Get keys from the first file
first_file = smooth_traj_files[0]
first_data = np.load(first_file, allow_pickle=True)
print("Keys in first file:", list(first_data.keys()))
for key in first_data.keys():
    merged_data[key] = {}

# Prepare lists to aggregate per traj and then stack at the end
obs_list = []
smoothed_obs_list = []
actions_list = []
rewards_list = []
terminals_list = []
masks_list = []
next_obs_list = []

# Now iterate through all files, populate the merged_data dict,
# and build arrays for observations, actions, rewards, etc. as we go
for filename in smooth_traj_files:
    data = np.load(filename, allow_pickle=True)
    basename = os.path.basename(filename)
    for key in data.keys():
        if key not in merged_data:
            merged_data[key] = {}
        merged_data[key][basename] = data[key]
    
    # Now process arrays for this trajectory
    obj_world = data['aruco_object_in_world']
    obj_ee = data['aruco_object_in_ee']
    ee_world = data['aruco_ee_in_world']


    smoothed_obj_world = data['smoothed_aruco_object_in_world']
    smoothed_obj_ee = data['smoothed_aruco_object_in_ee']
    smoothed_ee_world = data['smoothed_aruco_ee_in_world']

    states = data['states']
    actions = data['actions']
    n = actions.shape[0]
    # Sanity: all arrays have same length n
    assert obj_world.shape[0] == n
    assert obj_ee.shape[0] == n
    assert ee_world.shape[0] == n
    assert smoothed_obj_world.shape[0] == n
    assert smoothed_obj_ee.shape[0] == n
    assert smoothed_ee_world.shape[0] == n
    assert states.shape[0] == n

    # Each timestep: (obj_world (7), obj_ee (7), ee_world (7), last state dim) -> (22,)
    obs = np.concatenate([
        obj_world,
        obj_ee,
        ee_world,
        states[:, -1][:, None]
    ], axis=1)  # (n,22)
    obs_list.append(obs)

    smoothed_obs = np.concatenate([
        smoothed_obj_world,
        smoothed_obj_ee,
        smoothed_ee_world,
        states[:, -1][:, None]
    ], axis=1)  # (n,22)
    smoothed_obs_list.append(smoothed_obs)

    actions_list.append(actions)
    
    # Build rewards, terminals, masks
    rewards = -np.ones(n, dtype=np.float32)
    rewards[-1] = 0
    terminals = np.zeros(n, dtype=np.float32)
    terminals[-1] = 1
    masks = np.ones(n, dtype=np.float32)
    masks[-1] = 0

    rewards_list.append(rewards)
    terminals_list.append(terminals)
    masks_list.append(masks)

    # For next_observations, slide observations by 1 forward in this trajectory, last step same as last
    next_obs = np.empty_like(obs)
    next_obs[:-1] = obs[1:]
    next_obs[-1] = obs[-1]
    next_obs_list.append(next_obs)

# Stack all
observations = np.concatenate(obs_list, axis=0)  # (N, 22)
smoothed_observations = np.concatenate(smoothed_obs_list, axis=0)  # (N, 22)
actions = np.concatenate(actions_list, axis=0)
rewards = np.concatenate(rewards_list, axis=0)
terminals = np.concatenate(terminals_list, axis=0)
masks = np.concatenate(masks_list, axis=0)
next_observations = np.concatenate(next_obs_list, axis=0)

# Save results in merged_data-like dict for downstream code if needed
merged_data['observations'] = observations
merged_data['smoothed_observations'] = smoothed_observations
merged_data['actions_flat'] = actions  # avoid key conflict with dict-of-arrays actions
merged_data['rewards'] = rewards
merged_data['terminals'] = terminals
merged_data['next_observations'] = next_observations
merged_data['masks'] = masks

# Optionally print shapes for confirmation
print("observations", observations.shape)
print("smoothed_observations", smoothed_observations.shape)
print("actions", actions.shape)
print("rewards", rewards.shape)
print("terminals", terminals.shape)
print("next_observations", next_observations.shape)
print("masks", masks.shape)

Keys in first file: ['timestamps', 'states', 'actions', 'ee_poses_debug', 'object_pose', 'object_visible', 'aruco_ee_in_world', 'aruco_object_in_world', 'aruco_ee_in_object', 'aruco_object_in_ee', 'aruco_visibility', 'augmented_actions', 'metadata', 'smoothed_aruco_ee_in_world', 'smoothed_aruco_object_in_world', 'smoothed_aruco_ee_in_object', 'smoothed_aruco_object_in_ee']
observations (21163, 22)
smoothed_observations (21163, 22)
actions (21163, 7)
rewards (21163,)
terminals (21163,)
next_observations (21163, 22)
masks (21163,)


In [107]:
import numpy as np

np.savez("merged_data.npz", **merged_data)
print("merged_data saved to merged_data.npz")


merged_data saved to merged_data.npz


In [ ]:
data_path = '/home/steven/Desktop/work/research/action_chunk_q_learning/envs/hardware_old/merged_dataset.npz'
data = np.load(data_path, allow_pickle=True)
print(data.files)

for key in data.files:
    print(f"{key}: {data[key].shape}")



In [ ]:
data_path = '/home/steven/Desktop/work/research/action_chunk_q_learning/envs/hardware_old/merged_dataset.npz'
data = np.load(data_path, allow_pickle=True)
print(data.files)

for key in data.files:
    print(f"{key}: {data[key].shape}")



In [ ]:
data_path = '/home/steven/Desktop/work/research/action_chunk_q_learning/envs/hardware_old/merged_dataset.npz'
data = np.load(data_path, allow_pickle=True)
print(data.files)

for key in data.files:
    print(f"{key}: {data[key].shape}")



In [97]:
data_path = '/home/steven/Desktop/work/research/action_chunk_q_learning/envs/hardware_old/merged_dataset.npz'
data = np.load(data_path, allow_pickle=True)
print(data.files)

for key in data.files:
    print(f"{key}: {data[key].shape}")



['observations', 'actions', 'next_observations', 'rewards', 'terminals', 'masks']
observations: (21163, 27)
actions: (21163, 7)
next_observations: (21163, 27)
rewards: (21163,)
terminals: (21163,)
masks: (21163,)


In [119]:
# Load the first file, print its keys, then aggregate from all files under those keys.
merged_data_states = {}

if len(smooth_traj_files) == 0:
    raise RuntimeError("No trajectory files found!")

# Get keys from the first file
first_file = smooth_traj_files[0]
first_data = np.load(first_file, allow_pickle=True)
print("Keys in first file:", list(first_data.keys()))
for key in first_data.keys():
    merged_data_states[key] = {}

# Prepare lists to aggregate per traj and then stack at the end
obs_list = []
smoothed_obs_list = []
actions_list = []
rewards_list = []
terminals_list = []
masks_list = []
next_obs_list = []

# Now iterate through all files, populate the merged_data dict,
# and build arrays for observations, actions, rewards, etc. as we go
for filename in smooth_traj_files:
    data = np.load(filename, allow_pickle=True)
    basename = os.path.basename(filename)
    for key in data.keys():
        if key not in merged_data_states:
            merged_data_states[key] = {}
        merged_data_states[key][basename] = data[key]
    
    # Now process arrays for this trajectory
    obj_world = data['aruco_object_in_world']
    obj_ee = data['aruco_object_in_ee']
    ee_world = data['aruco_ee_in_world']

    smoothed_obj_world = data['smoothed_aruco_object_in_world']
    smoothed_obj_ee = data['smoothed_aruco_object_in_ee']
    smoothed_ee_world = data['smoothed_aruco_ee_in_world']
    
    ee_poses_debug = data['ee_poses_debug']
    
    states = data['states']
    actions = data['actions']
    n = actions.shape[0]
    # Sanity: all arrays have same length n
    assert obj_world.shape[0] == n
    assert obj_ee.shape[0] == n
    assert ee_world.shape[0] == n
    assert smoothed_obj_world.shape[0] == n
    assert smoothed_obj_ee.shape[0] == n
    assert smoothed_ee_world.shape[0] == n
    assert states.shape[0] == n

    # Each timestep
    obs = np.concatenate([
        obj_world,
        ee_poses_debug,
        states
    ], axis=1)
    obs_list.append(obs)

    smoothed_obs = np.concatenate([
        smoothed_obj_world,
        ee_poses_debug,
        states
    ], axis=1) 
    smoothed_obs_list.append(smoothed_obs)

    actions_list.append(actions)
    
    # Build rewards, terminals, masks
    rewards = -np.ones(n, dtype=np.float32)
    rewards[-1] = 0
    terminals = np.zeros(n, dtype=np.float32)
    terminals[-1] = 1
    masks = np.ones(n, dtype=np.float32)
    masks[-1] = 0

    rewards_list.append(rewards)
    terminals_list.append(terminals)
    masks_list.append(masks)

    # For next_observations, slide observations by 1 forward in this trajectory, last step same as last
    next_obs = np.empty_like(obs)
    next_obs[:-1] = obs[1:]
    next_obs[-1] = obs[-1]
    next_obs_list.append(next_obs)

# Stack all
observations = np.concatenate(obs_list, axis=0)  # (N, 22)
smoothed_observations = np.concatenate(smoothed_obs_list, axis=0)  # (N, 22)
actions = np.concatenate(actions_list, axis=0)
rewards = np.concatenate(rewards_list, axis=0)
terminals = np.concatenate(terminals_list, axis=0)
masks = np.concatenate(masks_list, axis=0)
next_observations = np.concatenate(next_obs_list, axis=0)

# Save results in merged_data-like dict for downstream code if needed
merged_data_states['observations'] = observations
merged_data_states['smoothed_observations'] = smoothed_observations
merged_data_states['actions_flat'] = actions  # avoid key conflict with dict-of-arrays actions
merged_data_states['rewards'] = rewards
merged_data_states['terminals'] = terminals
merged_data_states['next_observations'] = next_observations
merged_data_states['masks'] = masks

# Optionally print shapes for confirmation
print("observations", observations.shape)
print("smoothed_observations", smoothed_observations.shape)
print("actions", actions.shape)
print("rewards", rewards.shape)
print("terminals", terminals.shape)
print("next_observations", next_observations.shape)
print("masks", masks.shape)

Keys in first file: ['timestamps', 'states', 'actions', 'ee_poses_debug', 'object_pose', 'object_visible', 'aruco_ee_in_world', 'aruco_object_in_world', 'aruco_ee_in_object', 'aruco_object_in_ee', 'aruco_visibility', 'augmented_actions', 'metadata', 'smoothed_aruco_ee_in_world', 'smoothed_aruco_object_in_world', 'smoothed_aruco_ee_in_object', 'smoothed_aruco_object_in_ee']
observations (21163, 20)
smoothed_observations (21163, 20)
actions (21163, 7)
rewards (21163,)
terminals (21163,)
next_observations (21163, 20)
masks (21163,)


In [120]:
import numpy as np

np.savez("merged_data_states.npz", **merged_data_states)
print("merged_data_states saved to merged_data_states.npz")


merged_data_states saved to merged_data_states.npz
